In [6]:
import os
import numpy as np
import plotly.graph_objects as go

import gzip
import pickle as pkl

def loadall_results2(path, n_folds):
    regret = []
    history_actions = []
    history_outcomes = []
    perf = []
    with gzip.open(  path ,'rb') as f:
        for i in range(n_folds+1):
            try:
                data = pkl.load(f)
            except EOFError:
                break

            if type(data) == dict:
                regret.append( data['regret'] ) 
                history_actions.append( data['action_history'] )
                history_outcomes.append( data['outcome_history'] ) 
                perf.append( data['pred'] ) 

    
    return regret, history_actions, history_outcomes, perf

In [19]:
n_folds = 1
horizon = 9999
context = 'MNISTbinary'
model = 'MLP'
case = 'case1'
agent_name = 'EEneuralcbpside_v6' #ineural6, neuronal6

direct = './results/'
path = os.path.join(direct, '{}_{}_{}_{}_{}_{}.pkl.gz'.format(case,model,context,horizon,n_folds,agent_name) )
regret, action_history,outcome_history, perf = loadall_results2(path, n_folds)


In [7]:
n_folds = 25
horizon = 9999
model = 'LeNet'


material = {
    #'EEneuralcbpside_v5': {'color': [255, 255, 0], 'label': 'EEneuralcbpside_v5'},  # Red
    'EEneuralcbpside_v6': {'color': [255, 0, 0], 'label': 'Neural-CBP'},  # Red
    # 'ineural6': {'color': [51, 255, 255], 'label':'IneurAL (official)'},                    # Yellow
    # 'ineural3': {'color': [0, 0, 255], 'label':'IneurAL (tuned)'},                    # Cyan
    'neuronal6': {'color': [255, 0, 255], 'label':'Neuronal (official)'},                  # Magenta
    'neuronal3': {'color': [160, 160, 160], 'label':'Neuronal (tuned)'},                   # Orange
    # 'margin': {'color': [160, 160, 160], 'label':'Margin'},
    # 'cesa': {'color': [0, 0, 255], 'label':'Cesa'},
}

#  'random':{'color':[204,0,204],'label':'e-greedy'}

fig = go.Figure( )



data_models = {}
for model in ['LeNet']: #, 'LeNet'

    # if model == 'MLP':
    n_folds = 25
    datasets = [ 'MNIST', 'FASHION', 'CIFAR10', ]
    # else:
    #     datasets = ['MNIST', 'FASHION', 'CIFAR10',]
    #     n_folds = 25  'MNIST', 'FASHION', 'covertype', 'shuttle',

    data_regrets = {} 
    for data in datasets: 

        # if data in ['MNIST', 'FASHION', 'CIFAR10']:
        case = 'case2' 
        # elif data in ['covertype', 'shuttle']:
        #     case = 'game_case_seven'
        # else:
        # case = 'case1'
            
        final_regrets = {}
        for agent_name in material.keys():

            color, l_label = material[agent_name]['color'], material[agent_name]['label']

            r,g,b = color

            try:
                direct = './results/'
                path = os.path.join(direct, '{}_{}_{}_{}_{}_{}.pkl.gz'.format(case, model, data, horizon,n_folds,agent_name) )
                print(path)
                result, history_actions, history_outcomes, perf = loadall_results2(path, n_folds)
                result = np.array(result)
                # result = result[1:]
                result = result.astype(np.float32)
                # print(result.shape)
                final_regrets[l_label] = result[:,-1] 
            except:
                result = np.zeros((n_folds,horizon))

        if data == 'MNISTbinary':
            data = 'MNIST binary'
        if data == 'FASHION':
            data = 'Fashion'
            
        data_regrets[data] = final_regrets
        
    data_models[model] = data_regrets


# fig.show()
# fig.write_image("./figures/case1_{}_{}.pdf".format(model, context) )

./results/case2_LeNet_MNIST_9999_25_EEneuralcbpside_v6.pkl.gz
./results/case2_LeNet_MNIST_9999_25_neuronal6.pkl.gz
./results/case2_LeNet_MNIST_9999_25_neuronal3.pkl.gz
./results/case2_LeNet_FASHION_9999_25_EEneuralcbpside_v6.pkl.gz
./results/case2_LeNet_FASHION_9999_25_neuronal6.pkl.gz
./results/case2_LeNet_FASHION_9999_25_neuronal3.pkl.gz
./results/case2_LeNet_CIFAR10_9999_25_EEneuralcbpside_v6.pkl.gz
./results/case2_LeNet_CIFAR10_9999_25_neuronal6.pkl.gz
./results/case2_LeNet_CIFAR10_9999_25_neuronal3.pkl.gz


In [9]:
data_models

{'MLP': {'MNIST binary': {}, 'MagicTelescope': {}, 'adult': {}}}

In [8]:
import pandas as pd
# pd.DataFrame(data_regrets)
# data_regrets

samples = []

# Iterate over the dictionary to extract data
for model, dataset in data_models.items():
    for data, approach in dataset.items():
        for appr, values in approach.items():
            for val in values:
                samples.append([model, data, appr, val])

# Create a DataFrame
df = pd.DataFrame(samples, columns=['Model','Dataset', 'Approach', 'Value'])

# Calculate mean and standard error for each group
grouped = df.groupby(['Model', 'Dataset', 'Approach'])
mean = grouped['Value'].mean().reset_index()
std_error = grouped['Value'].std() / np.sqrt(grouped['Value'].count())
std_error = std_error.reset_index()

# Merge the mean and standard error into a single DataFrame
result = pd.merge(mean, std_error, on=['Model', 'Dataset', 'Approach'])
result.rename(columns={'Value_x': 'Mean', 'Value_y': 'StdError'}, inplace=True)

result['merge'] = result['Model'] + result['Approach']
result['merge2'] = result['Model'] + result['Dataset']

In [14]:
result

,Model,Dataset,Approach,Mean,StdError,merge,merge2
0,MLP,MNIST binary,Cesa,4853.439941,78.867845,MLPCesa,MLPMNIST binary
1,MLP,MNIST binary,IneurAL (official),3941.916748,44.711268,MLPIneurAL (official),MLPMNIST binary
2,MLP,MNIST binary,IneurAL (tuned),1726.959961,140.226192,MLPIneurAL (tuned),MLPMNIST binary
3,MLP,MNIST binary,Margin,3216.719971,290.599229,MLPMargin,MLPMNIST binary
4,MLP,MNIST binary,Neural-CBP,1351.920044,24.556257,MLPNeural-CBP,MLPMNIST binary
5,MLP,MNIST binary,Neuronal (official),1721.880005,9.509525,MLPNeuronal (official),MLPMNIST binary
6,MLP,MNIST binary,Neuronal (tuned),1721.439941,206.691284,MLPNeuronal (tuned),MLPMNIST binary
7,MLP,MagicTelescope,Cesa,3397.439941,144.331956,MLPCesa,MLPMagicTelescope
8,MLP,MagicTelescope,IneurAL (official),8876.280273,123.457436,MLPIneurAL (official),MLPMagicTelescope
9,MLP,MagicTelescope,IneurAL (tuned),5738.200195,125.035595,MLPIneurAL (tuned),MLPMagicTelescope


In [10]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import plotly.express as px
# Assuming df is your DataFrame and it has columns 'Dataset', 'Value', 'Model'

w = 0.1
def remove_substrings(s, substrings):
    for substring in substrings:
        s = s.replace(substring, '')
    return s


# approaches = result['merge'].unique()
# 'LeNetNeural-CBP', 'LeNetNeuronal (official)', 'LeNetNeuronal (tuned)',
# approaches = np.array([ 'MLPNeural-CBP', 'MLPNeuronal (official)','MLPNeuronal (tuned)', 'MLPIneurAL (official)', 
#                         'MLPIneurAL (tuned)',  'MLPCesa', 'MLPMargin' ], dtype=object) 
# approaches_names = [ remove_substrings(model, ['LeNet', 'MLP']) for model in approaches ]
# default_colors = px.colors.qualitative.Plotly
# model_colors = {model_n:col for model_n,col in zip(approaches_names,default_colors) }

approaches = {
    #'EEneuralcbpside_v5': {'color': [255, 255, 0], 'label': 'EEneuralcbpside_v5'},  # Red
    'LeNetNeural-CBP': {'color': [171, 99, 250], 'label': 'Neural-CBP'},  # Red
    # 'MLPIneurAL (official)': {'color': [255, 161, 90], 'label':'IneurAL (official)'},                    # Yellow
    # 'MLPIneurAL (tuned)': {'color': [25, 211, 243], 'label':'IneurAL (tuned)'},                    # Cyan
    'LeNetNeuronal (official)': {'color': [255, 102, 146], 'label':'Neuronal (official)'},                  # Magenta
    'LeNetNeuronal (tuned)': {'color': [182, 232, 128], 'label':'Neuronal (tuned)'},                   # Orange
    # 'MLPMargin': {'color': [255, 151, 255], 'label':'Margin'},
    # 'MLPCesa': {'color': [254, 203, 82], 'label':'Cesa'},
}


# Create an empty figure
fig = go.Figure()


views = []
for appr in approaches.keys():
    print(appr)
    appr_name = approaches[appr]['label']
    r,g,b = approaches[appr]['color']

    subset = result[result['merge'] == appr]
    dec = False
    if appr_name not in views:
        dec = True
        views.append(appr_name)
    

    if 'CBP' in appr_name:
        fig.add_trace(go.Bar(
            x=subset['merge2'],
            y=subset['Mean'],
            name=appr_name,
            error_y=dict(type='data', array=subset['StdError'], visible=True,thickness=1 ),
            marker_color='rgba({},{},{},1)'.format(r,g,b),marker_pattern_shape='x',marker_pattern_fgcolor='black', 
            showlegend = dec,
            # width=w
            ))
    else: 
        fig.add_trace(go.Bar(
            x=subset['merge2'],
            y=subset['Mean'],
            name=appr_name,
            error_y=dict(type='data', array=subset['StdError'], visible=True,thickness=1 ),
            marker_color='rgba({},{},{},1)'.format(r,g,b), 
            showlegend = dec,
            # width=w
            ))


# Add vertical black lines between data groups

datasets = result['merge2'].unique()
print(datasets)
for i, dataset in enumerate(datasets[:-1]):
    if i == 2:
        fig.add_vline(x=i + 0.5, line_width=2, line_color="black")
    else:
        fig.add_vline(x=i + 0.5, line_width=2, line_color="black",  line_dash="dot")


siz = 13

fig.update_layout(
    width=440,
    height=220,
    plot_bgcolor='white',  # Sets the plot background color
    paper_bgcolor='white',  # Sets the overall figure background color
    barmode='group',
    margin=dict(l=20, r=1, t=25, b=20),  # Small margins
    xaxis=dict(
        tickmode='array',
        tickvals=result['merge2'].unique(),
        ticktext = [ remove_substrings(model, ['LeNet', 'MLP']) for model in result['merge2'].unique() ],
        tickfont=dict(size=siz-2)  # Increase X-axis tick font size
    ),
    yaxis=dict(
        type="log",
        gridcolor='lightgrey',
        title="Final regret ± standard dev.",
        title_standoff=10,
        title_font=dict(size=siz),
        tickfont=dict(size=siz)  # Increase Y-axis tick font size
    ),
    showlegend=False

    # legend=dict(
    #     orientation="h",
    #     yanchor="bottom",
    #     y=-0.3,  # Adjust this value to position the legend
    #     xanchor="center",
    #     x=0.5,
    #     font=dict(size=siz)  # Increase legend font size
    # ),
)

fig.show()
fig.write_image("./figures/tst.pdf" )
fig.write_image("./figures/results_nonbinary_LeNet.pdf" )

LeNetNeural-CBP
LeNetNeuronal (official)
LeNetNeuronal (tuned)
['LeNetCIFAR10' 'LeNetFashion' 'LeNetMNIST']
